### Resource processing
This section deals with turning German resources into a spreadsheet which can be translated.

In [1]:
import spacy
nlp = spacy.load('de_core_news_sm')

In [2]:
import epub
from epub import epub2text
from collections import Counter, defaultdict
from functions import GermanLanguage, Word, Translation, partition_warnings, WarningLevel

In [3]:
de = GermanLanguage()

In [4]:
resource_dir = '/home/peter/Music/6000.German.e-book.Collection.epub'
books = os.listdir(resource_dir)[5:6]

resources = [] # type: List[str]
for i, book in enumerate(books):
    try:
        resource_fragments = epub.epub2text(f'{resource_dir}/{book}')
        resource = '\n'.join(resource_fragments)
        resources.append(resource)
    except KeyError:
        pass
doc = nlp('\n'.join(resources))

In [5]:
def simplify_pos(pos: str) -> str:
    return pos if pos == 'NOUN' or pos == 'VERB' else 'OTHER'

In [6]:
counts = Counter(
    (token.lemma_, simplify_pos(token.pos_))
        for token in doc
            if token.is_alpha and not token.is_stop and token.pos_ == 'VERB'
)
words = [
    Word(de, c[0], c[1], counts[c])
        for c in counts
            if de.is_valid_word(c[0], c[1])
]

In [7]:
test = '\n'.join([w.to_sheet() for w in sorted(words, key=lambda w: w.freq)])
with open('done.txt', 'w') as f:
    f.write(test)
print(test)

e", "en")	8
übertreffen	=GOOGLETRANSLATE("zu übertreffen", "de", "en")	8
versuchen	=GOOGLETRANSLATE("zu versuchen", "de", "en")	8
zuschanden	=GOOGLETRANSLATE("zu zuschanden", "de", "en")	8
betreffen	=GOOGLETRANSLATE("zu betreffen", "de", "en")	8
schick	=GOOGLETRANSLATE("zu schick", "de", "en")	8
widerfahren	=GOOGLETRANSLATE("zu widerfahren", "de", "en")	8
bilden	=GOOGLETRANSLATE("zu bilden", "de", "en")	8
vorstellen	=GOOGLETRANSLATE("zu vorstellen", "de", "en")	8
gelachet	=GOOGLETRANSLATE("zu gelachet", "de", "en")	8
zerbrechen	=GOOGLETRANSLATE("zu zerbrechen", "de", "en")	8
zusprechen	=GOOGLETRANSLATE("zu zusprechen", "de", "en")	8
erbarmen	=GOOGLETRANSLATE("zu erbarmen", "de", "en")	8
aufziehen	=GOOGLETRANSLATE("zu aufziehen", "de", "en")	8
aufhängen	=GOOGLETRANSLATE("zu aufhängen", "de", "en")	8
befleißen	=GOOGLETRANSLATE("zu befleißen", "de", "en")	8
fassen	=GOOGLETRANSLATE("zu fassen", "de", "en")	8
werden	=GOOGLETRANSLATE("zu werden", "de", "en")	8
verzeihen	=GOOGLETRANSLATE("zu 

### Spreadsheet corrections.

The resultant spreadsheet can have issues in it, so it's important to flag likely issues for manual overview.

In [3]:
with open('test.txt', 'r') as f:
    translations = [Translation.from_sheet(line) for line in f.read().splitlines()]
    by_warnings = partition_warnings(translations)

In [6]:
print([f'{t.source}: {t.dest}' for t in by_warnings[WarningLevel.NONSENSE]])

[]


In [59]:
from english_words import english_words_lower_set
import enchant
d = enchant.Dict('en')
enchant.list_languages()

['de_LU', 'en_GB', 'de_DE', 'de_BE', 'en_US', 'en_CA', 'en_AU', 'en']

In [63]:
# '' in english_words_lower_set
d.check('Cat')
# d.suggest('yeet')


True